In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import TransformerModel
from darts.metrics import smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

/home/eduardo/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity',
    'Soil_Moisture', 'Soil_Temperature', 
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}

In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
from sklearn.model_selection import TimeSeriesSplit

def objective(trial):
    input_chunk_length = trial.suggest_int("input_chunk_length", 24, 168, step=12)
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.05)
    batch_size = trial.suggest_int("batch_size", 64, 256, step=8)
    lr = trial.suggest_float("lr", 1e-6, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)


    nhead = trial.suggest_int('nhead', 2, 8)
    d_model = trial.suggest_int('d_model', 32, 256, step=8)
    if d_model % nhead != 0:
        raise optuna.exceptions.TrialPruned(f"d_model ({d_model}) must be divisible by nhead ({nhead})")
    num_encoder_layers = trial.suggest_int('num_encoder_layers', 1, 6)
    num_decoder_layers = trial.suggest_int('num_decoder_layers', 1, 6)
    activation = trial.suggest_categorical('activation', ["GLU", "Bilinear", "ReGLU", "GEGLU", "SwiGLU", "ReLU", "GELU"])
    
    early_stopper = EarlyStopping("val_loss", min_delta=0.0005, patience=15, verbose=True)
    prunner = PatchedPruningCallback(trial, monitor="val_loss")
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [early_stopper, prunner],
    }

    n_splits = 3
    tscv = TimeSeriesSplit(n_splits=n_splits)
    overall_smape_list = []
    fold = 0

    for train_index, val_index in tscv.split(df):
        print(f"\nFold {fold+1}/{n_splits}")
        train_df_fold = df.iloc[train_index]
        val_df_fold = df.iloc[val_index]
        train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
        val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

        scaler = Scaler()
        scaler = scaler.fit(train_fold)
        train_scaled = scaler.transform(train_fold)
        val_scaled = scaler.transform(val_fold)

        opt_kwargs = {"lr": lr, "weight_decay": weight_decay}

        model = TransformerModel(
            model_name=f"model_{fold+1}",
            input_chunk_length=input_chunk_length,
            output_chunk_length=24,
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            activation=activation,
            n_epochs=30,
            batch_size=batch_size,
            dropout=dropout,
            add_encoders=encoders,
            pl_trainer_kwargs=pl_trainer_kwargs,
            loss_fn=torch.nn.L1Loss(),
            optimizer_cls=torch.optim.Adam,
            lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
            lr_scheduler_kwargs={"mode": "min", "factor": 0.3, "patience": 7, "min_lr": 1e-7},
            save_checkpoints=True,
            force_reset=True,
            random_state=42,
            optimizer_kwargs=opt_kwargs,
        )
        model.fit(
            series=train_scaled,
            val_series=val_scaled,
            verbose=False,
            dataloader_kwargs={"num_workers": 11},
        )

        try:
            loaded_model = model.load_from_checkpoint(f"model_{fold+1}", best=True)
            print(f"Model loaded from checkpoint for trial {trial.number}, fold {fold}")
        except FileNotFoundError:
            print(f"Checkpoint not found. Using the in-memory trained model.")
            loaded_model = model

        forecasts = loaded_model.historical_forecasts(
            val_scaled,
            forecast_horizon=24,
            stride=1,
            retrain=False,
            verbose=False
        )

        overall_smape_val = smape(val_scaled, forecasts)
        print(f"SMAPE fold {fold}: {overall_smape_val}")
        overall_smape_list.append(overall_smape_val)
        fold += 1

    mean_smape = np.mean(overall_smape_list)
    print(f"Média dos SMAPE nos folds: {mean_smape}")

    trial_dict = {
        "trial_number": trial.number,
        "fold_smape": overall_smape_list,
        "mean_smape": mean_smape,
        "hyperparameters": trial.params
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return mean_smape if not np.isnan(mean_smape) else float("inf")

In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [8]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 11
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-06-15 12:23:01,788] A new study created in memory with name: no-name-6170d51e-e95e-4558-a809-ae236d85336b
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
/home/eduardo/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Fold 1/3


Metric val_loss improved. New best score: 12.912
Metric val_loss improved by 0.090 >= min_delta = 0.0005. New best score: 12.822
Metric val_loss improved by 0.056 >= min_delta = 0.0005. New best score: 12.766
Metric val_loss improved by 0.026 >= min_delta = 0.0005. New best score: 12.740
Metric val_loss improved by 0.022 >= min_delta = 0.0005. New best score: 12.718
Metric val_loss improved by 0.014 >= min_delta = 0.0005. New best score: 12.705
Metric val_loss improved by 0.010 >= min_delta = 0.0005. New best score: 12.695
Metric val_loss improved by 0.010 >= min_delta = 0.0005. New best score: 12.685
Metric val_loss improved by 0.008 >= min_delta = 0.0005. New best score: 12.677
Metric val_loss improved by 0.009 >= min_delta = 0.0005. New best score: 12.667
Metric val_loss improved by 0.010 >= min_delta = 0.0005. New best score: 12.657
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score: 12.654
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score

Model loaded from checkpoint for trial 0, fold 0


Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


...

SMAPE fold 2: 58.40508448133215
Média dos SMAPE nos folds: 68.6430062232998
Results of trial 73 saved in optuna_iteration_metrics/trial_73.json
Current value: 68.6430062232998, Current params: {'input_chunk_length': 24, 'dropout': 0.15000000000000002, 'batch_size': 120, 'lr': 0.0001048743847724991, 'weight_decay': 2.9856891086713972e-05, 'nhead': 8, 'd_model': 232, 'num_encoder_layers': 6, 'num_decoder_layers': 5, 'activation': 'GELU'}
Best value: 62.29155632988628, Best params: {'input_chunk_length': 72, 'dropout': 0.4, 'batch_size': 112, 'lr': 3.371095418689331e-05, 'weight_decay': 1.302793079680627e-06, 'nhead': 8, 'd_model': 240, 'num_encoder_layers': 1, 'num_decoder_layers': 3, 'activation': 'SwiGLU'}

Fold 1/3


[I 2025-06-15 17:04:51,001] Trial 74 pruned. Trial was pruned at epoch 0.
[I 2025-06-15 17:04:51,028] Trial 75 pruned. d_model (232) must be divisible by nhead (7)
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
/home/eduardo/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Current value: 12.196683422671029, Current params: {'input_chunk_length': 24, 'dropout': 0.1, 'batch_size': 136, 'lr': 3.5573742598032355e-05, 'weight_decay': 6.042903920076551e-05, 'nhead': 8, 'd_model': 216, 'num_encoder_layers': 6, 'num_decoder_layers': 5, 'activation': 'GELU'}
Best value: 62.29155632988628, Best params: {'input_chunk_length': 72, 'dropout': 0.4, 'batch_size': 112, 'lr': 3.371095418689331e-05, 'weight_decay': 1.302793079680627e-06, 'nhead': 8, 'd_model': 240, 'num_encoder_layers': 1, 'num_decoder_layers': 3, 'activation': 'SwiGLU'}
Current value: None, Current params: {'input_chunk_length': 24, 'dropout': 0.1, 'batch_size': 128, 'lr': 6.048148115509383e-05, 'weight_decay': 1.4264154570853813e-05, 'nhead': 7, 'd_model': 232}
Best value: 62.29155632988628, Best params: {'input_chunk_length': 72, 'dropout': 0.4, 'batch_size': 112, 'lr': 3.371095418689331e-05, 'weight_decay': 1.302793079680627e-06, 'nhead': 8, 'd_model': 240, 'num_encoder_layers': 1, 'num_decoder_layers

Metric val_loss improved. New best score: 12.154
Metric val_loss improved by 0.012 >= min_delta = 0.0005. New best score: 12.143
Metric val_loss improved by 0.019 >= min_delta = 0.0005. New best score: 12.124
Metric val_loss improved by 0.008 >= min_delta = 0.0005. New best score: 12.116
Metric val_loss improved by 0.006 >= min_delta = 0.0005. New best score: 12.110
Metric val_loss improved by 0.022 >= min_delta = 0.0005. New best score: 12.088
Metric val_loss improved by 0.009 >= min_delta = 0.0005. New best score: 12.079
Metric val_loss improved by 0.012 >= min_delta = 0.0005. New best score: 12.067
Metric val_loss improved by 0.004 >= min_delta = 0.0005. New best score: 12.063
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 12.062
Metric val_loss improved by 0.010 >= min_delta = 0.0005. New best score: 12.053
[I 2025-06-15 17:10:30,578] Trial 76 pruned. Trial was pruned at epoch 25.


Current value: 12.063115181240581, Current params: {'input_chunk_length': 36, 'dropout': 0.15000000000000002, 'batch_size': 112, 'lr': 0.00014087792115874672, 'weight_decay': 1.0475115870362078e-05, 'nhead': 8, 'd_model': 208, 'num_encoder_layers': 6, 'num_decoder_layers': 5, 'activation': 'GELU'}
Best value: 62.29155632988628, Best params: {'input_chunk_length': 72, 'dropout': 0.4, 'batch_size': 112, 'lr': 3.371095418689331e-05, 'weight_decay': 1.302793079680627e-06, 'nhead': 8, 'd_model': 240, 'num_encoder_layers': 1, 'num_decoder_layers': 3, 'activation': 'SwiGLU'}


In [9]:
print(f"Best SMAPE Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best SMAPE Value (Minimum): 62.29155632988628
Best Parameters: {'input_chunk_length': 72, 'dropout': 0.4, 'batch_size': 112, 'lr': 3.371095418689331e-05, 'weight_decay': 1.302793079680627e-06, 'nhead': 8, 'd_model': 240, 'num_encoder_layers': 1, 'num_decoder_layers': 3, 'activation': 'SwiGLU'}


In [10]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")

Best results saved in optuna_iteration_metrics/best_trial.json
